## Use background subtraction algorithm to find the key frames in lecture videos.

In [2]:
import cv2
import numpy as np
from PIL import Image
import os
import datetime
#from fpdf import FPDF

# 计算图像的感知哈希值
def calculate_hash(image):
    # 转换为灰度图像并调整大小为8x8像素
    if len(image) ==3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (8, 8))

    # 计算均值并二值化图像
    mean = np.mean(image)
    _, image = cv2.threshold(image, mean, 255, cv2.THRESH_BINARY)

    # 将图像转换为一维数组
    image = image.flatten()

    return image

# 计算汉明距离
def calculate_hamming_distance(hash1, hash2):
    return np.count_nonzero(hash1 != hash2)

# 判断两个图像是否相似
def is_duplicate(frame1, frame2, threshold):
    hash1 = calculate_hash(frame1)
    hash2 = calculate_hash(frame2)
    distance = calculate_hamming_distance(hash1, hash2)
    #print(distance)
    # 如果汉明距离小于阈值，则判定为重复帧
    if distance < threshold:
        return True
    else:
        return False

# 背景减法帧差
def frame_difference(frame, prev_frame):
    # 将帧转换为灰度图像
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if len(prev_frame.shape) == 2:
        prev_gray = prev_frame
        pass
    else:
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # 计算帧之间的绝对差异
    diff = cv2.absdiff(gray, prev_gray)

    # 进行形态学操作，去除噪声
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    diff = cv2.morphologyEx(diff, cv2.MORPH_OPEN, kernel)

    # 计算前景掩码
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    return thresh


# 使用OpenCV对背景像素进行统计建模
def background_subtraction(frame, bg_model):
    # 使用背景减法模型传递帧
    fg_mask = bg_model.apply(frame, learningRate=-1)

    # 计算前景百分比
    height, width = fg_mask.shape[:2]
    foreground_pixels = cv2.countNonZero(fg_mask)
    foreground_percent = (foreground_pixels / (width * height)) * 100

    return fg_mask, foreground_percent


# 视频到幻灯片转换处理（包括重复帧检测与删除）
def find_key_frames(video_path, output_path, frame_difference_threshold, bg_subtraction_threshold, duplicate_threshold):
    # 打开视频
    video = cv2.VideoCapture(video_path)

    # 读取第一帧
    _, prev_frame = video.read()
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # 创建概率背景减法模型
    bg_model = cv2.createBackgroundSubtractorMOG2()

    # 幻灯片计数器
    slide_counter = 1

    # 保存非重复帧的列表
    unique_frames = [prev_frame_gray]
    diff_frames=[]
    while True:
        # 读取当前帧
        ret, frame = video.read()
        if not ret:
            break
        
        timestamp_ms = video.get(cv2.CAP_PROP_POS_MSEC)
        timestamp_sec = timestamp_ms / 1000.0
        timestamp_str = str(datetime.timedelta(seconds=timestamp_sec)).split('.')[0].replace(':','_')

        # 背景减法帧差
        diff_frame = frame_difference(frame, prev_frame)
        diff_frames.append(diff_frame)

        # 判断是否保存帧
        if cv2.countNonZero(diff_frame) > frame_difference_threshold:
            cv2.imwrite(f"{output_path}/{slide_counter}_{timestamp_str}.jpg", frame)
            slide_counter += 1

        # 使用概率背景减法
        fg_mask, foreground_percent = background_subtraction(frame, bg_model)

        # 判断是否保存帧
        if foreground_percent > bg_subtraction_threshold:
            cv2.imwrite(f"{output_path}/{slide_counter}_{timestamp_str}.jpg", frame)
            slide_counter += 1

        # 更新前一帧
        prev_frame = frame.copy()
        prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

        # 判断是否为重复帧并进行删除
        is_duplicate_frame = False
        for unique_frame in unique_frames:
            if is_duplicate(prev_frame_gray, unique_frame, duplicate_threshold):
                is_duplicate_frame = True
                break

        if not is_duplicate_frame:
            unique_frames.append(prev_frame_gray)

    video.release()

# 调用函数进行视频到幻灯片转换处理
# find_key_frames("./videos/2   1   Distributed File Systems 15 50.mp4", "./key_frames/sb", 700, 1000, 5)


The algorithm takes time, so better to it in batches(by controlling the index of list 'videos')

In [3]:
video_path='./videos/'
videos=os.listdir(video_path)

In [4]:
for video in videos[90:]:
    output_path='./key_frames/'+video
    os.makedirs(output_path)
    find_key_frames(video_path+video, output_path, 700, 1000, 5)
    print(f'{video} finished!')

8   6   Exploiting Length 14 39 Advanced.mp4 finished!
8   7   Computing PageRank on Big Graphs 10 18 Advanced.mp4 finished!
8   8   Topic Specific PageRank 10 06.mp4 finished!
8   9   Application to Measuring Proximity in Graphs 6 25.mp4 finished!
